In [1]:
import pandas as pd
from pathlib import Path
import re
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径、常量和筛选条件"""
    # --- 【请修改】输入和输出路径 ---
    INPUT_DIR = Path(r'E:\负荷') # <--- 修改为存放每日文件的文件夹路径
    OUTPUT_FILE = Path(r'E:\行业负荷汇总.xlsx') # <--- 修改为你的输出文件路径

    # --- 文件匹配模式 ---
    FILENAME_PATTERN = r'国民经济 _ (\d{4}-\d{2}-\d{2})\.xlsx?'

    # --- Excel读取参数 ---
    SHEET_NAME = 0
    HEADER_ROWS = [0, 1] 
    
    # --- 【已优化】要筛选的行业 ---
    # 只需要提供你感兴趣的行业“序号”列表即可
    TARGET_INDUSTRY_INDEXES = [1, 2, 3, 4, 5, 6] # <--- 在这里添加你需要的其他行业序号

    # --- 数据提取范围 ---
    START_COL_NAME = '00:00'
    END_COL_NAME = '23:45'

# ==============================================================================
# 2. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数，完成数据提取、重组和保存"""
    print("--- 开始执行日度数据汇总任务 ---")

    if not Config.INPUT_DIR.exists():
        print(f"[致命错误] 输入文件夹不存在: {Config.INPUT_DIR}")
        return

    # 1. 查找所有符合条件的文件
    file_paths = sorted(Config.INPUT_DIR.glob('*.xls*'))
    
    valid_files = []
    for file_path in file_paths:
        if re.search(Config.FILENAME_PATTERN, file_path.name):
            valid_files.append(file_path)

    if not valid_files:
        print(f"[致命错误] 在文件夹 {Config.INPUT_DIR} 中没有找到任何符合命名规则的文件。")
        return
        
    print(f"发现 {len(valid_files)} 个有效文件，准备开始处理...")

    # 2. 【核心优化】从第一个有效文件中“学习”行业名称
    industry_mapping = {}
    print("  -> 正在从第一个文件中学习行业名称映射...")
    try:
        first_file_df = pd.read_excel(valid_files[0], sheet_name=Config.SHEET_NAME, header=Config.HEADER_ROWS)
        first_file_df.columns = first_file_df.columns.map(lambda x: x[0] if not str(x[0]).startswith('Unnamed') else x[1])
        
        # 确保'序号'和'名称'列存在
        if '序号' not in first_file_df.columns or '名称' not in first_file_df.columns:
            print("[致命错误] 第一个文件中未找到'序号'或'名称'列，无法自动学习行业名称。")
            return
            
        # 将序号设为索引以便快速查找
        first_file_df.set_index('序号', inplace=True)
        
        for index in Config.TARGET_INDUSTRY_INDEXES:
            if index in first_file_df.index:
                # 从'名称'列获取行业名
                industry_name = first_file_df.loc[index, '名称']
                industry_mapping[index] = industry_name
            else:
                print(f"  [警告] 在第一个文件中未找到序号为 {index} 的行业，将跳过此序号。")
        
        if not industry_mapping:
            print("[致命错误] 未能从第一个文件中学习到任何行业名称，流程终止。")
            return
            
        print("  -> 行业名称学习完成！将处理以下行业:")
        for index, name in industry_mapping.items():
            print(f"    - 序号 {index}: {name}")

    except Exception as e:
        print("[致命错误] 读取第一个文件并学习行业名称时出错。")
        print(traceback.format_exc())
        return

    # 3. 初始化数据存储字典
    industry_data_frames = {name: [] for name in industry_mapping.values()}

    # 4. 遍历所有文件进行数据提取
    for file_path in valid_files:
        date_str = re.search(Config.FILENAME_PATTERN, file_path.name).group(1)
        print(f"  -> 正在处理文件: {file_path.name} (日期: {date_str})")

        try:
            df = pd.read_excel(file_path, sheet_name=Config.SHEET_NAME, header=Config.HEADER_ROWS)
            df.columns = df.columns.map(lambda x: x[0] if not str(x[0]).startswith('Unnamed') else x[1])
            
            if '序号' not in df.columns:
                print(f"    [警告] 文件中未找到'序号'列，已跳过。")
                continue
            df.set_index('序号', inplace=True)

            try:
                start_col_idx = df.columns.get_loc(Config.START_COL_NAME)
                end_col_idx = df.columns.get_loc(Config.END_COL_NAME)
                time_point_columns = df.columns[start_col_idx : end_col_idx + 1]
            except KeyError as e:
                print(f"    [错误] 无法在文件中找到时间点列 '{e}'，已跳过。")
                continue

            # 使用学习到的映射来提取数据
            for index, name in industry_mapping.items():
                if index in df.index:
                    load_data = df.loc[index, time_point_columns]
                    new_row = pd.Series([date_str] + load_data.tolist(), index=['日期'] + list(time_point_columns))
                    industry_data_frames[name].append(new_row)
                else:
                    print(f"    [警告] 在文件中未找到序号为 {index} ('{name}') 的行。")

        except Exception as e:
            print(f"    [错误] 处理文件 {file_path.name} 时发生未知错误，已跳过。")
            print(traceback.format_exc())
            continue

    # 5. 保存结果到Excel
    print("\n所有文件处理完毕，正在生成最终的多工作表Excel报表...")
    Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(Config.OUTPUT_FILE, engine='openpyxl') as writer:
        for name, rows_list in industry_data_frames.items():
            if not rows_list:
                print(f"  -> 行业 '{name}' 未收集到任何数据，将生成空表。")
                header = ['日期'] + [f"{h:02d}:{m:02d}" for h in range(24) for m in range(0, 60, 15)]
                pd.DataFrame(columns=header).to_excel(writer, sheet_name=name, index=False)
                continue

            print(f"  -> 正在写入Sheet: {name}")
            final_df = pd.DataFrame(rows_list)
            final_df.to_excel(writer, sheet_name=name, index=False)
            
    print(f"\n--- 全部任务成功完成！结果已保存到: {Config.OUTPUT_FILE} ---")

if __name__ == '__main__':
    main()

--- 开始执行日度数据汇总任务 ---
发现 32 个有效文件，准备开始处理...
  -> 正在从第一个文件中学习行业名称映射...
  -> 行业名称学习完成！将处理以下行业:
    - 序号 1: 全社会用电总计
    - 序号 2: 全行业用电合计
    - 序号 3: 第一产业
    - 序号 4: 第二产业
    - 序号 5: 第三产业
    - 序号 6: 城乡居民生活用电合计
  -> 正在处理文件: 国民经济 _ 2025-11-20.xlsx (日期: 2025-11-20)
  -> 正在处理文件: 国民经济 _ 2025-11-21.xlsx (日期: 2025-11-21)
  -> 正在处理文件: 国民经济 _ 2025-11-22.xlsx (日期: 2025-11-22)
  -> 正在处理文件: 国民经济 _ 2025-11-23.xlsx (日期: 2025-11-23)
  -> 正在处理文件: 国民经济 _ 2025-11-24.xlsx (日期: 2025-11-24)
  -> 正在处理文件: 国民经济 _ 2025-11-25.xlsx (日期: 2025-11-25)
  -> 正在处理文件: 国民经济 _ 2025-11-26.xlsx (日期: 2025-11-26)
  -> 正在处理文件: 国民经济 _ 2025-11-27.xlsx (日期: 2025-11-27)
  -> 正在处理文件: 国民经济 _ 2025-11-28.xlsx (日期: 2025-11-28)
  -> 正在处理文件: 国民经济 _ 2025-11-29.xlsx (日期: 2025-11-29)
  -> 正在处理文件: 国民经济 _ 2025-11-30.xlsx (日期: 2025-11-30)
  -> 正在处理文件: 国民经济 _ 2025-12-01.xlsx (日期: 2025-12-01)
  -> 正在处理文件: 国民经济 _ 2025-12-02.xlsx (日期: 2025-12-02)
  -> 正在处理文件: 国民经济 _ 2025-12-03.xlsx (日期: 2025-12-03)
  -> 正在处理文件: 国民经济 _ 2025-12-04.xlsx (日期: 2025-12-04)